In [12]:
import pandas as pd
import sqlite3

In [13]:
connection = sqlite3.connect('./data/sql-murder-mystery.db')

def sql_query(query, c = connection):

    return pd.read_sql(query, c)


In [14]:
# Reporte policial del día en cuestion
query = '''
SELECT *
FROM crime_scene_report
where date=20180115 and city= "SQL City" and type="murder"
'''
df=sql_query(query)
df["description"][0]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [15]:
# Buscando a la direccion de los dos testigos y su nombre completo
query = '''

SELECT id, name, address_street_name, address_number
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'

UNION 

SELECT id, name, address_street_name, max (address_number)
FROM person
WHERE address_street_name = 'Northwestern Dr';


'''
df=sql_query(query)
df


,id,name,address_street_name,address_number
0,14887,Morty Schapiro,Northwestern Dr,4919
1,16371,Annabel Miller,Franklin Ave,103


In [26]:
# Obteniendo las declaraciones de ambos testigos.
query = '''

SELECT *
FROM interview
where person_id in (14887,16371)

'''
df = sql_query(query)

print (f'Declaracion de Morty: \n {df["transcript"][0]}') 
print (f'\nDeclaracion de Annabel: \n {df["transcript"][1]}')


Declaracion de Morty: 
 I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

Declaracion de Annabel: 
 I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


In [30]:
# Hombre : Miembro de oro del gym con número que empieza con 48Z
# Matricula del coche que se va incluye H42W
# El asesinato ocurrio el 9 de enero en el gimnasio coincidiendo con el turno de Annabel
#

query = '''

SELECT *

FROM get_fit_now_member G

INNER JOIN get_fit_now_check_in C ON G.id = C.membership_id
INNER JOIN person P ON P.id = G.person_id
INNER JOIN drivers_license L ON L.id = P.license_id

WHERE C.check_in_date = '20180109' 
AND G.membership_status = 'gold'
AND G.id LIKE '48Z%'
AND L.gender = 'male'
AND L.plate_number LIKE '%H42W%'

'''
df=sql_query(query)
df


# Asesino encontrado

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time,id,...,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700,67318,...,871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


In [31]:
# Seguimos buscando el jef@ de la trama
# Conocer más sobre la historia del asesino

query = '''

SELECT *
FROM interview
where person_id in (67318)

'''
df = sql_query(query)

print (f'Declaracion de Jeremy : \n {df["transcript"][0]}') 


Declaracion de Jeremy : 
 I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



In [33]:
# El asesino tiene relacion con: una mujer
# Tiene mucho dinero / Mide alrededor de 65-67 cm / Pelo rojo / Tesla Model / SQL Symphony 3 veces en diciembre 2017

query = '''
SELECT *
FROM person P
Inner Join drivers_license L on L.id = P.license_id
Inner Join income I on I.ssn = P.ssn
Inner Join facebook_event_checkin F on F.person_id = P.id

Where L.gender = "female" 
and L.car_make= "Tesla" 
and L.car_model = "Model S" 
and L.height <= 67
and L.hair_color = "red"


'''
df=sql_query(query)
df

,id,name,license_id,address_number,address_street_name,ssn,id,age,height,eye_color,...,gender,plate_number,car_make,car_model,ssn,annual_income,person_id,event_id,event_name,date
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,...,female,500123,Tesla,Model S,987756388,310000,99716,1143,SQL Symphony Concert,20171206
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,...,female,500123,Tesla,Model S,987756388,310000,99716,1143,SQL Symphony Concert,20171212
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,...,female,500123,Tesla,Model S,987756388,310000,99716,1143,SQL Symphony Concert,20171229
